# Model, predict and solve the problem.

Modeling data before training a machine learning (ML) model is crucial for various reasons. It involves understanding the data's characteristics, which aids in identifying potential issues and assessing its suitability for the ML task. Additionally, data modeling enables preprocessing steps like handling missing values, outlier detection, and feature scaling, which enhance data quality. Through feature engineering, new features can be created or existing ones transformed to improve the model's performance. Dimensionality reduction techniques reduce the number of features, improving efficiency and preventing overfitting. Modeling data facilitates model selection based on data characteristics and requirements. Finally, data modeling enables evaluation and validation, ensuring the model's reliability and generalization capabilities before deployment. Overall, data modeling is fundamental for optimizing data, improving model performance, and facilitating informed decision-making in ML.

## Libraries

In [ ]:
# Warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
%matplotlib inline

## Loading data

In [ ]:
data = pd.read_csv('/content/modeling-data.csv')

## Feature Engineering
Feature engineering is a fundamental process of transforming raw data to make it more suitable for machine learning algorithms. Its purpose is to select, create, and modify features (input variables) to enhance the performance of a machine learning model.

### Cardinality
The values of a categorical variable are selected from a group of categories, also called labels. For example, in the variable gender the categories are male and female, whereas in the variable city the labels could be London, Manchester, Brighton, and so on.

Categorical variables can contain different numbers of categories. The variable "gender" contains only 2 labels, but a variable like "city" or "postcode" can contain a huge number of labels.

The number of different labels is known as cardinality. A high number of labels within a variable is known as high cardinality. High cardinality poses the following challenges:

- Variables with too many labels tend to dominate those with only a few labels, particularly in decision tree-based algorithms.
- High cardinality may introduce noise.
- Some of the labels may only be present in the training data set and not in the test set, so machine learning algorithms may over-fit to the training set.
- Some labels may appear only in the test set, leaving the machine learning algorithms unable to perform a calculation over the new (unseen) observation.

let's check cardinality of the Categorical Features.

In [ ]:
for feature in data.select_dtypes(include = 'O').columns:

        print(f'{feature}: {data[feature].nunique()}')

print()
print(f"data len: {data.shape[0]}")

### Rare Labels

Categorical variables are those whose values are selected from a group of categories, also called labels. Labels may have different frequencies. Some categories appear a lot in the dataset, some others appear only in a small number of observations.

For example, in a dataset with information about loan applicants where one of the variables is "city" where the applicant lives, cities like "New York" may appear a lot in the data because New York has a huge population, whereas smaller towns like "Leavenworth" will appear only on a few occasions (population ~2000 people), because the population there is very small. A borrower is more likely to live in New York than in any other city because it has a bigger population.

More specifically,

- Rare categories can cause over-fitting, particularly in tree-based methods.

- Infrequent labels may add noise, which could cause over-fitting.

- Rare labels may be present only in training set, therefore causing over-fitting.

- Rare labels may appear only in the test set. Thus, the ML model will not know how to evaluate it.

**Note:** Sometimes rare values, are indeed important. For example, if we are building a model to predict fraudulent loan applications, which are by nature rare, then a rare value in a certain variable, may be indeed very predictive. This rare value could be telling us that the observation is most likely a fraudulent application, and therefore we should not to ignore it.

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(ncols = 4, nrows = 2, figsize = (10, 6))

axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8]

# For each categorical variable
for ax, feature in enumerate(data.select_dtypes(include = 'O').columns):

    pd.Series(data[feature].value_counts() / len(data)).sort_values(ascending = False).plot(kind = 'bar', ax = axes[ax])
    axes[ax].set_title(feature)
    axes[ax].axhline(y = 0.05, color = 'red')

plt.tight_layout()

**Festival**

In [ ]:
data = data.drop('Festival', axis = 1)

**City**

In [ ]:
data['City'] = data['City'].apply(lambda value: 'Urban' if value == 'Semi-Urban' else value)

### Data transformations

In [ ]:
import scipy.stats as stats

In [ ]:
ContinuousFeatures = data.drop(['Vehicle_condition', 'multiple_deliveries',
                               'OrderTime', 'Delivery_person_Age', 'Time_taken(min)'], axis = 1).select_dtypes(exclude = 'O').columns

ContinuousFeatures

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(ncols = 4, nrows = 2, figsize = (12, 5), sharey = True)

# Distribution with no transformations
sns.histplot(data = data, x = ContinuousFeatures[0], ax = ax1, bins = 20)
sns.histplot(data = data, x = ContinuousFeatures[1], ax = ax5, bins = 20)
ax1.set_title('No transformations')
ax1.set_xlabel('')
ax5.set_xlabel('')
ax1.set_ylabel(ContinuousFeatures[0])
ax5.set_ylabel(ContinuousFeatures[1])

# Yeo-Johnson transformation
temp1, param1 = stats.yeojohnson(data[ContinuousFeatures[0]].astype('float'))
temp2, param2 = stats.yeojohnson(data[ContinuousFeatures[1]].astype('float'))
sns.histplot(temp1, ax = ax2, bins = 20)
sns.histplot(temp2, ax = ax6, bins = 20)
ax2.set_title('Yeo-Johnson transformation')
ax2.set_xlabel('')
ax6.set_xlabel('')

# Box-Cox transformation
temp3, param3 = stats.boxcox(data[ContinuousFeatures[0]].astype('float'))
temp4, param4 = stats.boxcox(data[ContinuousFeatures[1]].astype('float'))
sns.histplot(temp3, ax = ax3, bins = 20)
sns.histplot(temp4, ax = ax7, bins = 20)
ax3.set_title('Box-Cox transformation')
ax3.set_xlabel('')
ax7.set_xlabel('')

# Log transformation
sns.histplot(np.log(data[ContinuousFeatures[0]]), ax = ax4, bins = 20)
sns.histplot(np.log(data[ContinuousFeatures[0]]), ax = ax8, bins = 20)
ax3.set_title('Log transformation')
ax3.set_xlabel('')
ax7.set_xlabel('');

### Feature Encoding
feature encoding is important to convert non-numeric data into a numerical representation that machine learning algorithms can process. It ensures algorithm compatibility, preserves relevant information, reduces dimensionality, handles non-numeric data, and ultimately improves model performance.

There is too many Encoding techniques: OneHot, Ordinal, Frequency, Ordered, Mean, Weight of evidence, etc... It's worth noting that there is no one-size-fits-all encoding method, and the choice of encoding technique depends on the specific characteristics of the data, the machine learning algorithms being used, and the goals of the analysis. It is often helpful to experiment with different encoding approaches and evaluate their impact on the model's performance before deciding which one is most suitable for a particular classification problem.

For this particular case I have decided to use: **OneHot Encoding**.

In [ ]:
data.drop('Time_taken(min)', axis = 1).columns

In [ ]:
data = pd.get_dummies(data, drop_first = True)

### X, y

You can think of X as the input data, and y as the corresponding output or target values that you are trying to predict or model using machine learning algorithms. The goal is to learn a function or a relationship between X and y, so that given new input data, the model can make accurate prediction.

In [ ]:
# Reseting Index

data.reset_index(drop = True, inplace = True)

In [ ]:
X = data.drop('Time_taken(min)', axis = 1)
y = data['Time_taken(min)']

**Feature engineering notes:**

-  Cardinality in all categorical variables is controlled, being order_day the feature with more cardinality (7)
-  City and Festival both had a label that only appeared few times (Festival: Yes, City Semi-Urban) For festival I decided to remove the entire feature from the training dataset, since deliting "Yes" mean that there will be only "No" in the variable, this means having no predicting value. For city I decided to map those Semi-Urban as Urban.
-  There is no transformations that can be applied really satisfies the normal distribution so linear models need to be avoided.
-  Data was encoded using OneHot encoding since this is a regression problem.


## Performance & Error Metrics

Using performance and error metrics is essential for evaluating machine learning models. These metrics provide a quantitative assessment of model performance, aiding in model selection and hyperparameter tuning. They help detect overfitting or underfitting and enable performance monitoring over time. Metrics also play a crucial role in decision-making processes, allowing stakeholders to assess the feasibility and effectiveness of deploying ML models in real-world settings. Performance and error metrics provide a systematic and objective approach to evaluate and optimize ML models for optimal performance and decision-making.

The metrics I am gonna take into consideration are:

- **Mean Squared Error (MSE):** MSE is a common metric used to measure the average squared difference between the predicted and actual values in regression problems. It quantifies the overall quality of a regression model by assessing how closely its predictions align with the true values.

- **Root Mean Squared Error (RMSE):** RMSE is a commonly used evaluation metric in regression problems. It is derived from the Mean Squared Error (MSE) and provides a measure of the average magnitude of the prediction errors made by a regression model. To calculate RMSE, you take the square root of the MSE.
- **Mean Absolute Error (MAE):** MAE provides a measure of the average magnitude of the prediction errors made by the model. Unlike MSE and RMSE, which involve squaring the errors, MAE considers the absolute differences, which makes it less sensitive to outliers or instances with larger errors. The advantage of MAE is its simplicity and ease of interpretation. It represents the average absolute deviation from the true values and is expressed in the same units as the target variable.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def GetMetrics(y, predictions):
    return {'MSE' : mean_squared_error(y, predictions),
            'RMSE' : np.sqrt(mean_squared_error(y, predictions)),
            'MAE': mean_absolute_error(y, predictions)}

## Model Selection

For this particular problem I have chosen to use a LSTM NN. A Long Short-Term Memory (LSTM) is a type of recurrent neural network (RNN) architecture that is widely used for processing sequential and time series data. LSTMs are designed to overcome the limitations of traditional RNNs in capturing long-term dependencies and preserving information over longer sequences.

LSTMs are particularly effective in tasks where there are time dependencies and contextual information matters. They have been successfully applied in various domains, including natural language processing, speech recognition, machine translation, and time series forecasting.

The key feature of an LSTM is its memory cell, which allows it to selectively remember or forget information over time. It consists of several gates that control the flow of information:

- **Forget Gate:** Determines which information to discard from the previous cell state.
- **Input Gate:** Decides which new information to update and add to the cell state.
- **Output Gate:** Filters the information from the current cell state to produce the output.
The LSTM's ability to selectively retain or discard information through these gates enables it to capture long-term dependencies and handle vanishing or exploding gradients during training.

LSTMs are trained using backpropagation through time (BPTT), an extension of backpropagation where the gradient is calculated across time steps. This allows the network to learn and update its parameters by minimizing a specific loss function, such as mean squared error (MSE) or mean absolute error (MAE), depending on the regression task.

By leveraging the memory cell and gates, LSTMs can model complex sequences and make predictions based on historical context. They have become a popular choice for sequential data analysis and have significantly contributed to advancements in various fields where temporal relationships are essential.

## Training keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':

  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
model = Sequential()
model.add(Dense(27, activation = 'relu'))
model.add(Dense(13, activation = 'relu'))
model.add(Dense(6, activation = 'relu'))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.fit(x = X_train, y = y_train, validation_data = (X_test, y_test),
          batch_size = 1, epochs = 100, callbacks = [early_stop])

In [ ]:
# Making predictions
predictions = model.predict(X_test)

In [ ]:
Summary = pd.DataFrame(GetMetrics(y = y_test, predictions = predictions), index = ['Score'])

In [ ]:
Summary

In [ ]:
predictions

In [ ]:
ComparationTable = pd.DataFrame({
    'Real Value' : y_test.values,
    'Model Prediction' : [round(item[0]) for item in predictions],
    'Difference' : y_test.values - [round(item[0]) for item in predictions],
    'Difference %' : np.absolute((y_test.values - [round(item[0]) for item in predictions]) / y_test.values * 100)})

ComparationTable.head()

In [ ]:
ComparationTable.describe()[1:]

In [ ]:
# Our predictions
plt.scatter(y_test, predictions)

# Perfect predictions
plt.plot(y_test, y_test, 'r')
plt.title('Model Predictions vs. Perfect Predictions (Line)');

In [ ]:
errors = y_test.values.reshape(4387, 1) - predictions
sns.distplot(errors)
plt.title('Residuals');

In [ ]:
# Worst predictions
ComparationTable.sort_values(by = 'Difference %', ascending = False).head(20)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, nrows = 1, figsize = (10, 3))
sns.histplot(data = ComparationTable, x = 'Difference %', ax = ax1)
ax1.set_xlim(left = 0)
ax1.set_title('Predictions Difference % to real value')

sns.boxplot(data = ComparationTable, x = 'Difference %', ax = ax2)
ax2.set_title('Predictions Difference % to real value');